<a href="https://colab.research.google.com/github/huiseung02/huiseung_DTSC3020_Fall2025/blob/main/Assignment5_Ch_10%2611.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment5: CRM Cleanup @ **DalaShop**
*Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions*  
.....

**Total: 3 points**  (Two questions, 1.5 pts each)  

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.

## Scenario
You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).  
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.

## Data file (given by the company): `contacts_raw.txt`
Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.

In [1]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")

Wrote contacts_raw.txt with sample DalaShop data.


## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex
Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).  
   - Trim whitespace with `strip()` before checking.  
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).  
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.  
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).  
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**
- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling  
- (0.5) Correct email regex validation + filtering  
- (0.4) Phone normalization + case-insensitive de-dup (keep first)  
- (0.2) Clean code, clear names, minimal docstrings/comments

In [21]:
import re
from pathlib import Path
import csv

def read_contacts(filename):
    path = Path(filename)
    try:
        with path.open(encoding="utf-8") as f:
            return f.readlines()
    except FileNotFoundError:
        print(f"File '{filename}' not found.")
        return []

def validate_email(email):
    email = email.strip()
    if re.fullmatch(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", email):
        return email
    return ""

def normalize_phone(phone):
    phone = re.sub(r"\D", "", phone)
    if len(phone) >= 10:
        return phone[-10:]
    else:
        return ""

def main():
    input_file = "contacts_raw.txt"
    output_file = "contacts_clean.csv"

    lines = read_contacts(input_file)
    if not lines:
        return

    cleaned_rows = []
    seen_emails = set()

    for line in lines:
        parts = [p.strip() for p in line.split(",")]

        if len(parts) == 2:
            name_email, phone_raw = parts
            match = re.search(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", name_email)
            if not match:
                continue
            email = match.group(0)
            name = re.sub(r"<.*?>", "", name_email).strip()

        elif len(parts) == 3:
            name, email, phone_raw = parts

        else:
            continue

        email = validate_email(email)
        if not email:
            continue

        email_key = email.casefold()
        if email_key in seen_emails:
            continue
        seen_emails.add(email_key)

        phone = normalize_phone(phone_raw)
        cleaned_rows.append([name.strip(), email, phone])

    with open(output_file, "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["name", "email", "phone"])
        writer.writerows(cleaned_rows)

    print(f"Cleaned contacts written to {output_file}")

if __name__ == "__main__":
    main()

Cleaned contacts written to contacts_clean.csv


## Q2 (1.5 pts) — Unit testing with `unittest`
Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.  
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.  
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).  
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).




In [22]:
# Write your answer
import unittest
import re

class TestCRMCleanup(unittest.TestCase):
    def test_validate_email_valid(self):
        valid_cases = [
            "user@example.com",
            "john.doe@domain.co",
            "alice_bob123@company.org",
            "USER+test@Example.NET"
        ]
        for email in valid_cases:
            self.assertEqual(validate_email(email), email.strip())

    def test_validate_email_invalid(self):
        invalid_cases = [
            "user@@example.com",
            "userexample.com",
            "wrong@domain",
            "user@.com",
            "@missinguser.com"
        ]
        for email in invalid_cases:
            self.assertEqual(validate_email(email), "")

    def test_normalize_phone(self):
        self.assertEqual(normalize_phone("(469) 555-1234"), "4695551234")
        self.assertEqual(normalize_phone("972-555-8888"), "9725558888")
        self.assertEqual(normalize_phone("+1 214 555 9999"), "2145559999")
        self.assertEqual(normalize_phone("12345"), "")
        self.assertEqual(normalize_phone("001234567890"), "1234567890")

    def test_parsing(self):
        data = """Alice <alice@example.com>, (469) 555-1234
Bob <bob@example.org>, +1 972-555-8888"""
        lines = data.strip().split("\n")
        rows = []
        for line in lines:
            name_email, phone = line.split(",")
            name_email = name_email.strip()
            phone = phone.strip()
            match = re.search(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", name_email)
            if not match:
                continue
            email = match.group(0)
            name = re.sub(r"<.*?>", "", name_email).strip()
            phone = normalize_phone(phone)
            rows.append((name, email, phone))
        expected = [
            ("Alice", "alice@example.com", "4695551234"),
            ("Bob", "bob@example.org", "9725558888")
        ]
        self.assertEqual(rows, expected)

    def test_deduplication(self):
        data = [
            "Alice <alice@example.com>, (469) 555-1234",
            "Bob <bob@example.org>, +1 972-555-8888",
            "Duplicate <ALICE@EXAMPLE.COM>, 111-111-1111"
        ]
        seen = set()
        rows = []
        for line in data:
            name_email, phone = line.split(",")
            name_email = name_email.strip()
            phone = phone.strip()
            match = re.search(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", name_email)
            if not match:
                continue
            email = match.group(0)
            name = re.sub(r"<.*?>", "", name_email).strip()
            email_key = email.casefold()
            if email_key in seen:
                continue
            seen.add(email_key)
            phone = normalize_phone(phone)
            rows.append((name, email, phone))
        expected = [
            ("Alice", "alice@example.com", "4695551234"),
            ("Bob", "bob@example.org", "9725558888")
        ]
        self.assertEqual(rows, expected)

if __name__ == "__main__":
    import unittest
    unittest.main(argv=[''], exit=False)


.....
----------------------------------------------------------------------
Ran 5 tests in 0.008s

OK


## Grading rubric (total 3 pts)
- **Q1 (1.5 pts)**  
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling  
  - (0.5) Email validation (regex + strip + full match) and filtering  
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)  
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**  
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)  
  - (0.6) Parsing & de-dup tests that assert exact expected rows  
  - (0.3) Standard `unittest` structure and readable test names
